In [1]:
from keras.applications import MobileNet

Using TensorFlow backend.


In [2]:
model = MobileNet(weights='imagenet'  , include_top = False ,input_shape=(224,224,3))

In [3]:
model.layers[0].input

<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>

In [4]:
model.layers[0].__class__.__name__

'InputLayer'

In [5]:
model.layers[3].trainable

True

In [6]:
model.layers[3].trainable = False

In [7]:
model.layers[3].trainable

False

In [8]:
for  layer in model.layers:
    layer.trainable = False

In [9]:
model.layers[9].trainable

False

In [10]:
model.layers

In [11]:
model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32

In [12]:
from keras.models import Sequential

In [13]:
from keras.layers import Dense, GlobalAveragePooling2D

In [14]:
top_model=model.output

In [15]:
top_model = GlobalAveragePooling2D()(top_model)

In [16]:
top_model = Dense(2048, activation='relu')(top_model)

In [17]:
top_model = Dense(1024, activation='relu')(top_model)

In [18]:
top_model = Dense(512, activation='relu')(top_model)

In [19]:
top_model = Dense(256, activation='relu')(top_model)

In [20]:
top_model = Dense(5, activation='softmax')(top_model)

In [21]:
from keras.models import Model

In [22]:
newmodel= Model(inputs=model.input,outputs=top_model)

In [23]:
newmodel.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [24]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'CelebData/train/'
validation_data_dir = 'CelebData/val/'

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True)
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 


In [25]:
img_rows, img_cols = 224, 224 

In [26]:
batch_size = 32
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')

Found 93 images belonging to 5 classes.
Found 25 images belonging to 5 classes.


In [27]:
nb_train_samples=93
nb_validation_samples=25
newmodel.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
newmodel.fit(
     train_generator,
    steps_per_epoch = 50,
    epochs = 7,
    validation_data = validation_generator,
    validation_steps = 1)

Epoch 1/7
50/50 [==============================] - 177s 4s/step - loss: 0.5368 - accuracy: 0.8076 - val_loss: 2.0068 - val_accuracy: 0.5600
Epoch 2/7
50/50 [==============================] - 170s 3s/step - loss: 0.0411 - accuracy: 0.9852 - val_loss: 0.8178 - val_accuracy: 0.7600
Epoch 3/7
50/50 [==============================] - 159s 3s/step - loss: 0.0240 - accuracy: 0.9910 - val_loss: 0.6585 - val_accuracy: 0.7600
Epoch 4/7
50/50 [==============================] - 161s 3s/step - loss: 0.0733 - accuracy: 0.9761 - val_loss: 2.3151 - val_accuracy: 0.6800
Epoch 5/7
50/50 [==============================] - 160s 3s/step - loss: 0.0540 - accuracy: 0.9852 - val_loss: 0.5304 - val_accuracy: 0.8400
Epoch 6/7
50/50 [==============================] - 159s 3s/step - loss: 9.6193e-05 - accuracy: 1.0000 - val_loss: 1.0374 - val_accuracy: 0.8800
Epoch 7/7
50/50 [==============================] - 159s 3s/step - loss: 9.6958e-06 - accuracy: 1.0000 - val_loss: 1.0349 - val_accuracy: 0.8800


In [28]:
newmodel.save('face_recognition_mobilenet.h5')

In [1]:
from keras.models import load_model
classifier = load_model('face_recognition_mobilenet.h5')

Using TensorFlow backend.


In [2]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

CelebData_dict =     {"[0]": "ben_afflek"    , 
                      "[1]": "elton_john"    ,
                      "[2]": "jerry_seinfeld",
                      "[3]": "madonna"       ,
                      "[4]": "mindy_kaling"  }

CelebData_dict_n =    {"ben_afflek": "ben_afflek"    , 
                       "elton_john": "elton_john"    ,
                       "jerry_seinfeld": "jerry_seinfeld",
                       "madonna": "madonna"       ,
                       "mindy_kaling": "mindy_kaling"  }

def draw_test(name, pred, im):
    face = CelebData_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, face, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + CelebData_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,10):
    input_im = getRandomImage("CelebData/val/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)

cv2.destroyAllWindows()

Class - jerry_seinfeld
Class - jerry_seinfeld
Class - madonna
Class - jerry_seinfeld
Class - ben_afflek
Class - elton_john
Class - madonna
Class - elton_john
Class - jerry_seinfeld
Class - madonna
